<u><font size="6">**Anghami to Spotify Transfer**</font></u>

<font size="5"><u>Instructions</u></font>
<font size="4">
    
1- Open anghami web on the playlist(or downloads and likes) you want to copy and scroll to the bottom so that all the songs in that playlist are loaded and visible 

2- Right click and save as Html file and remeber its path

3- Go to https://developer.spotify.com/ and go to dashboard and register your account as a developper account

4- Create a new app after registering you account as a developper account 4

5- After creating the app copy your client id, client secret(you might need to click on show client secret to see it)

6- Click on edit settings and set the redirect url to http://127.0.0.1:8080 

7- Go to your spotify account and copy your account username 

8- Populate the variables in the following cell with the information you just copied and run all 
</font>

In [ ]:
html_path ="*populate here*" #the anghami playlist saved html file path
client_id = "*populate here*" #the client id you copied in step 4
client_secret = "*populate here*" #the client secret you copied in step 4
redirect_url = "http://127.0.0.1:8080"
username = "*populate here*" #your spotify username you copied in step 7
spotify_playlist_name = "*populate here*" #name for the new playlist you're creating on spotify to copy music to 
save_to_text = True #set to False if you don't want to export the playlist to a txt file
txt_save_path = "*populate here*" #the path you want to export the txt playlist to
txt_song_artist_seperator = ' ' #the seperator between the song nam and the artist in the saved txt file 

## basic installations and imports

In [ ]:
pip install beautifulsoup4
pip install spotipy

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import spotipy
import json
from spotipy.oauth2 import SpotifyOAuth

## Reading and parsing HTML file

In [ ]:
 with open(html_path ,encoding="utf8") as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')

In [ ]:
classLst=["cell cell-title","cell cell-title marquee"]
mydivs = soup.find_all("div", class_=classLst)
songs=[]
print("Reading Songs")
for job_element in tqdm(mydivs):
    title_element = job_element.find("span")
    songs.append(title_element.text)
mydivsart = soup.find_all("div", {"class": "cell cell-artist"})
artists=[]
print("Reading Artists")
for job_element in tqdm(mydivsart):
    artists.append(job_element.text)
if len(songs) != len(artists):
    print("Reading Erorr")
else:
    print("Reading successful")

<font size="4">**Optional** Print the list</font>

In [ ]:
for i in range(len(songs)):
    print(songs[i]+ txt_song_artist_seperator +artists[i])

<font size="4">Write To File</font>

In [ ]:
if save_to_text:
    with open(txt_save_path, 'w',encoding="utf8") as fp:
        for i in tqdm(range(len(songs))):
            # write each item on a new line
            fp.write(songs[i]+txt_song_artist_seperator+artists[i]+'\n')
        print('Done')

In [ ]:
scope='playlist-modify-public'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_url,
                                               scope=scope,
                                              username=username))
sp.user_playlist_create(user=username,name=spotify_playlist_name,public=True,description="imported from Anghami",)

In [ ]:
notfound=[]
playlist=sp.user_playlists(user=username)['items'][0]['id']
for i in tqdm(range(len(songs))):
    try:
        res=sp.search(q=songs[i]+" "+artists[i],type='track',limit=1)
    except:
        notfound.append(songs[i]+" "+artists[i])
    else:
        if len(res['tracks']['items'])==0:
            notfound.append(songs[i]+" "+artists[i])
        else:
            uri=res['tracks']['items'][0]['uri']
            sp.user_playlist_add_tracks(user=username,playlist_id=playlist,tracks=[uri])

        